In [ ]:
import pandas as pd
import numpy as np
import skopt
import sys
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import joblib
import json
import os

In [ ]:
training=pd.read_csv("/path_to_ADNI_training_dataset.csv")
test=pd.read_csv("/path_to_ADNI_test_dataset.csv")

In [ ]:
datasetAIBL=pd.read_csv("/path_to_AIBL_dataset.csv")
datasetOASIS=pd.read_csv("/path_to_OASIS_dataset.csv")

In [ ]:
training.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
test.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetAIBL.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)
datasetOASIS.rename(columns={ "Unnamed: 0": "PTID" }, inplace = True)

In [ ]:
training=training.replace([np.inf, -np.inf], np.nan)
test=test.replace([np.inf, -np.inf], np.nan)
datasetAIBL=datasetAIBL.replace([np.inf, -np.inf], np.nan)
datasetOASIS=datasetOASIS.replace([np.inf, -np.inf], np.nan)

In [ ]:
training=training.dropna(axis=1, how='all')

In [ ]:
def forward_selection(cls,trainDsX, trainDsY,validDsX,validDsY):
    initial_features = trainDsX.columns.tolist()
    best_features = []
    act_accuracy=0
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features,dtype=np.float64)
        for new_column in remaining_features:
            model = cls.fit(trainDsX[best_features+[new_column]],trainDsY)
            predValid=cls.predict(validDsX[best_features+[new_column]])
            new_pval[new_column] = accuracy_score(validDsY, predValid)
        maxaccuracy = new_pval.max()
        if(maxaccuracy>act_accuracy):
            best_features.append(new_pval.idxmax())
            act_accuracy=maxaccuracy
        else:
            break
    return best_features

In [ ]:
modelParams=["rf","xgbTree","rpart","svmPoly","svmRBF","logReg"]
featureSet=["MRI+SocioDemography","MRI+SocioDemography+Genetics","MRI+SocioDemography+Genetics+kogTest"]
fsMethods=["yes","no"]

In [ ]:
filenameCSV="/path_to_results_file.csv"

In [ ]:
for featureSetSelected in featureSet:
    for model in modelParams:
        for selectMethod in fsMethods:
            if(featureSetSelected=="MRI+SocioDemography"):
                X_train=training.drop(["separation","DX","APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE"], axis=1)
                LE = LabelEncoder()
                LE.fit(training["DX"])
                y_train=LE.transform(training["DX"])
                X_test=test.drop(["separation","DX","APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE"], axis=1)
                y_test=LE.transform(test["DX"])
                y_AIBL=LE.transform(datasetAIBL["DX"])
                y_OASIS=LE.transform(datasetOASIS["DX"])
            elif(featureSetSelected=="MRI+SocioDemography+Genetics"):
                X_train=training.drop(["separation","DX","LDELTOTAL","LIMMTOTAL","MMSCORE"], axis=1)
                LE = LabelEncoder()
                LE.fit(training["DX"])
                y_train=LE.transform(training["DX"])
                X_test=test.drop(["separation","DX","LDELTOTAL","LIMMTOTAL","MMSCORE"], axis=1)
                y_test=LE.transform(test["DX"])
                y_AIBL=LE.transform(datasetAIBL["DX"])
                y_OASIS=LE.transform(datasetOASIS["DX"])
            elif(featureSetSelected=="MRI+SocioDemography+Genetics+kogTest"):
                X_train=training.drop(["separation","DX"], axis=1)
                LE = LabelEncoder()
                LE.fit(training["DX"])
                y_train=LE.transform(training["DX"])
                X_test=test.drop(["separation","DX"], axis=1)
                y_test=LE.transform(test["DX"])
                y_AIBL=LE.transform(datasetAIBL["DX"])
                y_OASIS=LE.transform(datasetOASIS["DX"])
            train_MRI=training.drop(["separation","APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'], axis=1)
            test_MRI=training.drop(["separation","APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'], axis=1)
            trainValidSplit=splitDataset.splitTrainingValidationAndTestdataset(train_MRI,Percent_Test=0.2,Percent_Valid=0.0,groupVar="DX",seed=101)


            trainValidSplit=trainValidSplit.set_index('PTID')
            
            is_training_split=(trainValidSplit.separation.isin(["training"]))
            training_MRI=trainValidSplit[is_training_split]
            training_MRI=training_MRI.drop(["separation"],axis=1)
            training_MRI_X=training_MRI.drop(["DX"], axis=1)
            training_MRI_y=LE.transform(training_MRI["DX"])
            is_test_split=(trainValidSplit.separation=="test")
            test_MRI=trainValidSplit[is_test_split]
            test_MRI=test_MRI.drop(["separation"],axis=1)
            test_MRI_X=test_MRI.drop(["DX"], axis=1)
            test_MRI_y=LE.transform(test_MRI["DX"])
            numeric_features = training_MRI_X.columns.values.tolist()

            numeric_transformer = Pipeline(
                steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
            )

            categorical_features = []
            categorical_transformer = SimpleImputer(strategy='most_frequent')

            preprocessor = ColumnTransformer(
                transformers=[
                    ("num", numeric_transformer, numeric_features),
                    ("cat", categorical_transformer, categorical_features),
                ]
            )

            clf = preprocessor
            clf.fit(training_MRI_X)
            training_MRI_X_data=pd.DataFrame(clf.transform(training_MRI_X),columns=training_MRI_X.columns)
            test_MRI_X_data=pd.DataFrame(clf.transform(test_MRI_X),columns=training_MRI_X.columns)
            if(selectMethod=="yes"):
                if (model=="rf"):
                    cls=RandomForestClassifier(random_state=42)
                if (model=="xgbTree"):
                    cls=xgb.XGBClassifier(booster ="gbtree",n_jobs=5,random_state=42)
                if (model=="rpart"):
                    cls=DecisionTreeClassifier(random_state=42)
                if (model=="svmPoly"):
                    cls=SVC(kernel="poly",random_state=42,probability=True)
                if (model=="svmRBF"):
                    cls=SVC(kernel="rbf",random_state=42,probability=True)
                if (model=="logReg"):
                    cls= LogisticRegression(random_state=42)
                selectedFeatures=forward_selection(cls,training_MRI_X_data, training_MRI_y,test_MRI_X_data,test_MRI_y)
            if(selectMethod=="no"):
                selectedFeatures=training_MRI_X_data.columns.values.tolist()



            X_train_MRI=X_train.filter(selectedFeatures,axis=1)
            X_train_Rest=X_train.filter(["APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'],axis=1)

            X_test_MRI=X_test.filter(selectedFeatures,axis=1)
            X_test_Rest=X_test.filter(["APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'],axis=1)

            X_AIBL_MRI=datasetAIBL.filter(selectedFeatures,axis=1)
            X_AIBL_Rest=datasetAIBL.filter(["APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'],axis=1)
            
            X_OASIS_MRI=datasetOASIS.filter(selectedFeatures,axis=1)
            X_OASIS_Rest=datasetOASIS.filter(["APOE4","LDELTOTAL","LIMMTOTAL","MMSCORE","AGE","PTEDUCAT",'PTGENDER_Female'],axis=1)

            
            X_train_filtered = pd.concat([X_train_Rest.reset_index(drop=True), X_train_MRI.reset_index(drop=True)], axis=1)
            X_test_filtered = pd.concat([X_test_Rest.reset_index(drop=True), X_test_MRI.reset_index(drop=True)], axis=1)
            X_AIBL_filtered = pd.concat([X_AIBL_Rest.reset_index(drop=True), X_AIBL_MRI.reset_index(drop=True)], axis=1)
            X_OASIS_filtered = pd.concat([X_OASIS_Rest.reset_index(drop=True), X_OASIS_MRI.reset_index(drop=True)], axis=1)
            #RandomForest
            if(model=="rf"):
                params = dict()
                params['clf__n_estimators'] = skopt.space.space.Integer(250,1250)
                params['clf__max_features'] = skopt.space.space.Integer(2,X_train_MRI.shape[1])
                params['clf__min_samples_leaf'] = skopt.space.space.Integer(1,20)

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', RandomForestClassifier(random_state=42))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)

            if(model=="xgbTree"):
                params = dict()
                params['clf__n_estimators'] = skopt.space.space.Integer(1,500)
                params['clf__max_depth'] = skopt.space.space.Integer(1,20)
                params['clf__learning_rate'] = skopt.space.space.Real(10e-10,1,prior="log-uniform",base=10)
                params['clf__gamma'] = skopt.space.space.Real(0,20)
                params['clf__min_child_weight'] = skopt.space.space.Real(1,30)
                params['clf__subsample'] = skopt.space.space.Real(0,1)
                params['clf__colsample_bytree'] = skopt.space.space.Real(0,1)

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', xgb.XGBClassifier(booster ="gbtree",random_state=42))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)
            if model=="rpart":
                params = dict()
                params['clf__criterion'] = skopt.space.space.Categorical(['gini', 'entropy'])
                params['clf__splitter'] = skopt.space.space.Categorical(["best","random"])
                params['clf__max_depth'] = skopt.space.space.Integer(1,100)
                params['clf__min_samples_split'] = skopt.space.space.Real(0.01,1)

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', DecisionTreeClassifier(random_state=42))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)

            if model=="svmPoly":
                params = dict()
                params['clf__C'] = skopt.space.space.Real(10e-4,10e2,prior="log-uniform",base=10)
                params['clf__degree'] = skopt.space.space.Integer(1,10)
                params['clf__gamma'] = skopt.space.space.Categorical(["scale","auto"])

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', SVC(kernel="poly",random_state=42,probability=True))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)

            if model=="svmRBF":
                params = dict()
                params['clf__C'] = skopt.space.space.Real(10e-4,10e2,prior="log-uniform",base=10)
                params['clf__gamma'] = skopt.space.space.Categorical(["scale","auto"])

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', SVC(kernel="rbf",random_state=42,probability=True))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)
            if model=="logReg":
                params = dict()
                params['clf__C'] = skopt.space.space.Real(10e-4,10e2,prior="log-uniform",base=10)
                params['clf__penalty'] = skopt.space.space.Categorical(["l2", "none"])

                numeric_features = X_train_filtered.columns.values.tolist()
                numeric_transformer = Pipeline(
                [
                    ('imputer', SimpleImputer(strategy="median")),
                    ('scaler', StandardScaler())
                ]
                )

                preprocessor = ColumnTransformer(
                [
                    ('numericals', numeric_transformer, numeric_features)
                ],
                remainder = 'drop'
                )

                pipeline = Pipeline(
                [
                    ('preprocessing', preprocessor),
                    ('clf', LogisticRegression(random_state=42))
                ]
                )


                rskf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 10, random_state = 42)

                cv = BayesSearchCV(estimator=pipeline,n_iter=25,search_spaces= params, cv = rskf, n_jobs = -1,optimizer_kwargs={'initial_point_generator': "lhs","n_initial_points":10,"acq_func": "LCB"},random_state=50,return_train_score=True ,scoring='accuracy')
                cv.fit(X_train_filtered, y_train)
            joblib.dump(cv, '/path_to_save_model/model_'+featureSetSelected+"_"+model+"_"+selectMethod+'.pkl', compress = 1)
            pred=cv.predict(X_test_filtered)
            pred_prob=cv.predict_proba(X_test_filtered)
            accADNI=np.round_(accuracy_score(y_test, pred),decimals=4)*100
            BACCADNI=np.round_(balanced_accuracy_score(y_test, pred),decimals=4)*100
            AUROCADNI=np.round_(roc_auc_score(y_test, pred_prob[:,1],average="macro"),decimals=4)*100
            F1ADNI=np.round_(f1_score(y_test, pred, average='macro'),decimals=4)*100
            MCCADNI=np.round_(matthews_corrcoef(y_test, pred),decimals=3)
            
            predAIBL=cv.predict(X_AIBL_filtered)
            pred_prob_AIBL=cv.predict_proba(X_AIBL_filtered)
            accAIBL=np.round_(accuracy_score(y_AIBL, predAIBL),decimals=4)*100
            BACCAIBL=np.round_(balanced_accuracy_score(y_AIBL, predAIBL),decimals=4)*100
            AUROCAIBL=np.round_(roc_auc_score(y_AIBL, pred_prob_AIBL[:,1],average="macro"),decimals=4)*100
            F1AIBL=np.round_(f1_score(y_AIBL, predAIBL, average='macro'),decimals=4)*100
            MCCAIBL=np.round_(matthews_corrcoef(y_AIBL, predAIBL),decimals=3)
            
            predOASIS=cv.predict(X_OASIS_filtered)
            pred_prob_OASIS=cv.predict_proba(X_OASIS_filtered)
            accOASIS=np.round_(accuracy_score(y_OASIS, predOASIS),decimals=4)*100
            BACCOASIS=np.round_(balanced_accuracy_score(y_OASIS, predOASIS),decimals=4)*100
            AUROCOASIS=np.round_(roc_auc_score(y_OASIS, pred_prob_OASIS[:,1],average="macro"),decimals=4)*100
            F1OASIS=np.round_(f1_score(y_OASIS, predOASIS, average='macro'),decimals=4)*100
            MCCOASIS=np.round_(matthews_corrcoef(y_OASIS, predOASIS),decimals=3)
            
            sdCV=np.round_(cv.cv_results_['std_test_score'][cv.best_index_],decimals=4)*100 

            meanCV=np.round_(cv.cv_results_['mean_test_score'][cv.best_index_],decimals=4)*100
            d = {"selectMethod":[selectMethod],"featureSetSelected":[featureSetSelected],'model': [model],'params': [json.dumps(cv.best_params_)], 'Mean CV Accuracy':[meanCV], 'Sd CV Accuracy':[sdCV], "ACCADNI":[accADNI], "BACCADNI":[BACCADNI], "AUROCADNI":[AUROCADNI], "F1ADNI":[F1ADNI], "MCCADNI":[MCCADNI], "ACCAIBL":[accAIBL], "BACCAIBL":[BACCAIBL], "AUROCAIBL":[AUROCAIBL], "F1AIBL":[F1AIBL], "MCCAIBL":[MCCAIBL], "ACCOASIS":[accOASIS], "BACCOASIS":[BACCOASIS], "AUROCOASIS":[AUROCOASIS], "F1OASIS":[F1OASIS], "MCCOASIS":[MCCOASIS]}
            df = pd.DataFrame(data=d)
            if os.path.isfile(filenameCSV):
                df.to_csv(filenameCSV, mode='a', header=False)
            else:
                df.to_csv(filenameCSV, mode='w', header=True)